In [3]:
import pandas as pd
import os
import glob
import numpy as np
import statistics as stats
import scipy
import src.mpra_tools.predicted_occupancy as po
import src.mpra_tools.fasta_utils as fu
from src.grammar.sentences import sentences_Cspacer
import math
import matplotlib.pyplot as plt
from collections import Counter, defaultdict
import random

In [14]:
activity_df = pd.read_csv("Data/activity.csv", index_col=0)
motif_file = "Data/Motifs/summarized_motifs.parquet"

In [15]:
labels = activity_df.index.to_list()

In [16]:
sentences = sentences_Cspacer(labels,motif_file)

118364 sentences to create
	 29591 / 118364 sentences created
	 59182 / 118364 sentences created
	 88773 / 118364 sentences created
All sentences created


In [15]:
a = np.array([1,2,3])
random.choice(a)

1